In [1]:
import pandas as pd
import re
import pickle


from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from nltk.corpus import stopwords
from autocorrect import Speller
import sklearn
import numpy as np
import matplotlib as plt
import pycaret
import nltk
import spacy

from sklearn.tree import DecisionTreeClassifier
nltk.download('stopwords')

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.22.


[nltk_data] Downloading package stopwords to C:\Users\Embross
[nltk_data]     Employee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pickle
from simpletransformers.classification import ClassificationModel

#loading classification model, 17 categories
with open('./model-2.pickle','rb') as f:
    model2 = pickle.load(f)

#loading punitive model, 1 or 0
with open('./model-1-v1.pickle','rb') as f:
    model1 = pickle.load(f)


In [3]:
#Functions used in this script

# created but can assume these companies spelled it right in there TOA
def spelling(text):
    
    spell = Speller(lang='en')
    return str(spell(text))
    
def tokening(text):
    
    #nltk.download('punkt')
    tokenized_word=word_tokenize(text)
    return tokenized_word
    
def removing_stopwords(text):
    
    #nltk.download('punkt')
    tokenized_word1=tokening(text)
    #nltk.download('stopwords')
    stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for w in tokenized_word1:
        if w not in stop_words:
            filtered_sent.append(w)
    return " ".join(filtered_sent)



def Lemmating(text):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    tokenized_word1=tokening(text)
    stemmed_words=[]
    for w in tokenized_word1:
        stemmed_words.append(lemmatizer.lemmatize(w))
    return " ".join(stemmed_words)

def pos_tag(text):
    #nltk.download('averaged_perceptron_tagger')
    tokenized_word1=tokening(text)
    pos=nltk.pos_tag(removing_stopwords(text1))
    return pos
    
# function called for cleaning html tags from text
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [4]:
def preprocessingSentence(sentence):
    #remove all tags like <p> etc. in the data frame "data_sentenced"
    sentence = cleanhtml(sentence)
    # remove any non alpha numeric
    sentence = re.sub('[^0-9a-zA-Z]+', ' ', sentence)
    #remove stop words
    sentence = removing_stopwords(sentence)
    #Lemmaing words
    sentence = Lemmating(sentence)
    return sentence  
    
#punitive model, to see whether the sentence is punitive or not
def punitivePredict(sentences):
    #input list of sentence, get predict value back from model
    prediction = model1.predict(sentences)[0]
    #return list of result matches with input order
    return prediction

#classification model based on 17 categories
def classifyPredict(sentences):
    #input list of sentence, get predict value back from model
    prediction = model2.predict(sentences)[0]
    #return list of result matches with input order
    return prediction

#creating company vectors, number of rows = len(sentences), number of columns = number of categories
#if the sentence is punitive, then modifies the value
#if the sentence is not punitive, then do not change that row
def createCompanyVectors(punitivePredictions, classifyPredictions):
    companyVector = np.full((len(punitivePredictions), 17), 0)
    for i in range(len(punitivePredictions)):
        if punitivePredictions[i] == 1:
            companyVector[i][classifyPredictions[i]] = 1

    return companyVector

#calculating the cosine similarity
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity 
    """
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    return dot_product / (norm_a * norm_b)

In [5]:
# import data
with open('WeChatTC.txt', 'r') as f:
    paragraph = f.read()

#split to sentence from paragrah
sentences = nltk.sent_tokenize(paragraph)
preprocessedSentences = []
result = []
#result.append('sentence')

In [6]:
#preprocessing the sentence by defined functions
for i in range(len(sentences)):
    preprocessedSentence = preprocessingSentence(sentences[i])
    preprocessedSentences.append(preprocessedSentence)

In [7]:
#get punitive result
punitivePredictions = punitivePredict(preprocessedSentences)
#get classification result
classifyPredictions = classifyPredict(preprocessedSentences)


In [8]:
#get company vector based on punitive result and classification result
company_vector = createCompanyVectors(punitivePredictions, classifyPredictions)

In [9]:
#get company ethics vector from company vector
company_ethics_vector = np.ones((len(punitivePredictions), 17))- company_vector
company_ethics_vector

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [10]:
#Normalized company ethics vector - get avergae value
company_ethics_vector = company_ethics_vector.mean(axis=0)

# User comes in

In [11]:
#get user vector based on 17 categories
user_vector = np.array([0, 0, 0,5,0,10,0,0,0,0,0,0,0,0,0,0,0])
# normalize by diving array by 10 which is the highest number
user_vector = user_vector/10

In [14]:
#how close between user preference and company vector - cosine similarity
print('Cosine Similarity : {:.2f}%'.format(cos_sim(company_ethics_vector, user_vector)*100))

Cosine Similarity : 30.26%
